<a href="https://colab.research.google.com/github/emegnahc/quant/blob/main/quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

On dw llama.cpp

In [ ]:
!git clone https://github.com/ggml-org/llama.cpp

On le build

In [ ]:
!cd llama.cpp && cmake -B build -DGGML_CUDA=ON && cmake --build build --config Release

Install des requirements / peut-être un redem de kernel et une réinstall du repo llama.cpp...

In [ ]:
!pip install -r llama.cpp/requirements.txt

On choisi le modèle qui va bien. Si ressource suffisante sinon, un plus petit de la même famille pour rester au plus prêt

In [ ]:
MODEL_ID = "Qwen/Qwen3-Coder-480B-A35B-Instruct"
MODEL_NAME = MODEL_ID.split("/")[-1]

On récup le modèle depuis HF

In [ ]:
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

A essayer en convertissant d'abord en fp16 voir en skippant ce step...
Voir si possible et si résultat différent

In [ ]:
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/examples/convert_legacy_llama.py {MODEL_NAME} --outtype f16 --outfile {fp16}

On crée la matrice d'importance qui n'est pas une page mais qui s'obtient en inférent sur le dataset de "calibration".
A retrouver la syntaxe exacte sur le repo llama.cpp


In [ ]:
!./llama.cpp/build/bin/llama-imatrix

On choisi de la quantif 4bits étant donné ce que l'on peut attendre des perfs cf. certains tests réalisés par des utilisateurs.
Il faut ajouter le fichier créé à l'étape précédente

In [ ]:
method = "IQ4_XS"
qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{METHOD}.gguf"
!./llama.cpp/build/bin/llama-quantize {fp16} {qtype} {METHOD} --imatrix TO_BE_COMPLETED

Ensuite on évalue avec llama-perplexity. A compléter la cellule ci-dessous cf.doc sur le repo llama.cpp avec les options pour avoir le plus de détails

In [ ]:
!./llama.cpp/build/bin/llama-perplexity